# Winbindex Metadata File Analysis

In [ ]:
import pathlib
import pandas
import json
data = pathlib.Path("../../cvedata/data/", 'winbindex-fileinfo-pandas.json.gz')

In [ ]:
df = pandas.read_json(data)
df.head()

In [ ]:
# Drop invalid entries
total_rows = df.shape[0]
df = df[df['timestamp'].notna()]
df = df[df['virtualSize'].notna()]
df = df[df['version'].notna()]

invalid_rows_total = total_rows - df.shape[0]
invalid_rows_total

In [ ]:
# clean up version
df['version'] = df['version'].apply(lambda x: x.split()[0])

In [ ]:
def get_microsoft_download_url(row):

    timestamp = format(int(row['timestamp']), '08X')
    virtual_size = format(int(row['virtualSize']),'X')
    filename = row['filename']
    
    return f'https://msdl.microsoft.com/download/symbols/{filename}/{timestamp}{virtual_size}/{filename}'

In [ ]:
df['url'] = df.apply(get_microsoft_download_url,axis=1)

In [ ]:
url_df = df.groupby(by='url').aggregate(list)
url_df['collisions'] = url_df['version'].apply(lambda x: len(x))


In [ ]:
url_df

In [ ]:
url_df[(url_df['collisions'] > 1) & (url_df['filename'].str.contains('rpcrt4'))].sort_values(by='collisions', ascending=False)

In [ ]:
url_df[(url_df['collisions'] > 1)]

In [ ]:
dupes = pandas.DataFrame.duplicated(df, ['url']) 
#dup_df = dup_df[dup_df[0] == True]
dupes_df = df[dupes].set_index('filename').sort_index()


In [ ]:
df[dupes].groupby(by='filename')['url'].count().sort_values(ascending=False)